In [86]:
#Standard Header used on the projects

#first the major packages used for math and graphing
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
import scipy.special as sp
import pandas as pd

#Custome graph format style sheet
#plt.style.use('Prospectus.mplstyle')

#If being run by a seperate file, use the seperate file's graph format and saving paramaeters
#otherwise set what is needed
if not 'Saving' in locals():
    Saving = True
if not 'Titles' in locals():
    Titles = True
if not 'Ledgends' in locals():
    Ledgends = True
if not 'FFormat' in locals():
    FFormat = '.png'
Computer = "SciClone"   # "SciClone"    "WinLap"  "LinLap"   "Desktop" 

#Standard cycle for collors and line styles
default_cycler = (cycler('color', ['0.00', '0.40', '0.60', '0.70']) + cycler(linestyle=['-', '--', ':', '-.']))
plt.rc('axes', prop_cycle=default_cycler)
my_cmap = plt.get_cmap('gray')

In [87]:
PlotWidthIn = 11
PlotHeightIn = 3.75
PlotDPI = 150

In [88]:
import datetime

In [89]:
import multiprocessing
from joblib import Parallel, delayed

import pywt
from pywt._extensions._pywt import (DiscreteContinuousWavelet, ContinuousWavelet,
                                Wavelet, _check_dtype)
from pywt._functions import integrate_wavelet, scale2frequency 

In [90]:
from scipy.optimize import curve_fit

In [91]:
WaveletToUse = 'gaus2'
scales = 400
spacer = 5

In [92]:
if Computer == "SciClone":
    location = '/sciclone/home20/dchendrickson01/image/'
elif Computer == "WinLap":
    location = 'C:\\Data\\'
elif Computer == "Desktop":
    location = "E:\\Backups\\Dan\\CraneData\\"
elif Computer == "LinLap":
    location = '/home/dan/Output/'
    

if Computer ==  "SciClone":
    rootfolder = '/sciclone/home20/dchendrickson01/'
    folder = '/sciclone/data10/dchendrickson01/SmallCopy/'
elif Computer == "Desktop":
    rootfolder = location
    folder = rootfolder + "SmallCopy\\"
elif Computer =="WinLap":
    rootfolder = location
    folder = rootfolder + "SmallCopy\\"   
elif Computer == "LinLap":
    rootfolder = '/home/dan/Data/'
    folder = rootfolder + 'SmallCopy/'

In [93]:
SensorPositonFile = rootfolder + 'SensorStatsSmall.csv'
SaveModelFolder = rootfolder + 'SavedModel\\'
OutputVectors = np.genfromtxt(open(SensorPositonFile,'r'), delimiter=',',skip_header=1,dtype=int, missing_values=0)


In [94]:
#Extra Headers:
import os as os
import statistics as st
import os as os
import pandas as pd
import random
import multiprocessing
from joblib import Parallel, delayed
import time

In [95]:
#Columns names for a file with all 6 dimmensions
Header = np.array(['t', 's','x','y','z','s'])

In [96]:
def cwt_fixed(data, scales, wavelet, scalespace =spacer, sampling_period=1.):
    """
    COPIED AND FIXED FROM pywt.cwt TO BE ABLE TO USE WAVELET FAMILIES SUCH
    AS COIF AND DB

    COPIED From Spenser Kirn
    
    All wavelet work except bior family, rbio family, haar, and db1.
    
    cwt(data, scales, wavelet)

    One dimensional Continuous Wavelet Transform.

    Parameters
    ----------
    data : array_like
        Input signal
    scales : array_like
        scales to use
    wavelet : Wavelet object or name
        Wavelet to use
    sampling_period : float
        Sampling period for frequencies output (optional)

    Returns
    -------
    coefs : array_like
        Continous wavelet transform of the input signal for the given scales
        and wavelet
    frequencies : array_like
        if the unit of sampling period are seconds and given, than frequencies
        are in hertz. Otherwise Sampling period of 1 is assumed.

    Notes
    -----
    Size of coefficients arrays depends on the length of the input array and
    the length of given scales.

    Examples
    --------
    >>> import pywt
    >>> import numpy as np
    >>> import matplotlib.pyplot as plt
    >>> x = np.arange(512)
    >>> y = np.sin(2*np.pi*x/32)
    >>> coef, freqs=pywt.cwt(y,np.arange(1,129),'gaus1')
    >>> plt.matshow(coef) # doctest: +SKIP
    >>> plt.show() # doctest: +SKIP
    ----------
    >>> import pywt
    >>> import numpy as np
    >>> import matplotlib.pyplot as plt
    >>> t = np.linspace(-1, 1, 200, endpoint=False)
    >>> sig  = np.cos(2 * np.pi * 7 * t) + np.real(np.exp(-7*(t-0.4)**2)*np.exp(1j*2*np.pi*2*(t-0.4)))
    >>> widths = np.arange(1, 31)
    >>> cwtmatr, freqs = pywt.cwt(sig, widths, 'mexh')
    >>> plt.imshow(cwtmatr, extent=[-1, 1, 1, 31], cmap='PRGn', aspect='auto',
    ...            vmax=abs(cwtmatr).max(), vmin=-abs(cwtmatr).max())  # doctest: +SKIP
    >>> plt.show() # doctest: +SKIP
    """

    # accept array_like input; make a copy to ensure a contiguous array
    dt = _check_dtype(data)
    data = np.array(data, dtype=dt)
    if not isinstance(wavelet, (ContinuousWavelet, Wavelet)):
        wavelet = DiscreteContinuousWavelet(wavelet)
    if np.isscalar(scales):
        scales = np.r_[1:scales+1] * scalespace
    if data.ndim == 1:
        try:
            if wavelet.complex_cwt:
                out = np.zeros((np.size(scales), data.size), dtype=complex)
            else:
                out = np.zeros((np.size(scales), data.size))
        except AttributeError:
            out = np.zeros((np.size(scales), data.size))
        precision = 10
        int_psi, x = integrate_wavelet(wavelet, precision=precision)
        step = x[1] - x[0]
        for i in np.arange(np.size(scales)):
            
            j = np.floor(
                np.arange(scales[i] * (x[-1] - x[0]) + 1) / (scales[i] * step))
            if np.max(j) >= np.size(int_psi):
                j = np.delete(j, np.where((j >= np.size(int_psi)))[0])
            coef = - np.sqrt(scales[i]) * np.diff(np.convolve(data, int_psi[j.astype(int)][::-1]))
            d = (coef.size - data.size) / 2.
            out[i, :] = coef[int(np.floor(d)):int(-np.ceil(d))]
        #frequencies = scale2frequency(wavelet, scales, precision)
        #if np.isscalar(frequencies):
        #    frequencies = np.array([frequencies])
        #for i in np.arange(len(frequencies)):
        #    frequencies[i] /= sampling_period
        frequencies = []
        return out, frequencies
    else:
        raise ValueError("Only dim == 1 supported")



In [97]:
def getWvltImage(data, wvt=WaveletToUse, ns=scales, scalespace=spacer):
    if np.shape(data)[0] == 2:
        wvt = data[1]
        data = data[0]

    #try:
    cfX, freqs = cwt_fixed(data[0], ns, wvt,scalespace)
    cfY, freqs = cwt_fixed(data[1], ns, wvt,scalespace)
    cfZ, freqs = cwt_fixed(data[2], ns, wvt,scalespace)
    
    ThreeColor = np.concatenate((cfX,cfY),axis=2)
    ThreeColor = np.concatenate((ThreeColor,cfZ),axis=2)
    
    minVal = np.min(ThreeColor)
    maxVal = np.max(ThreeColor)
    rangeVal = maxVal - minVal#

    ThreeColor -= minVal
    ThreeColor /= rangeVal

    ThreeColor *= float(255)

    ThreeColor = ThreeColor.astype(int)
    
    return ThreeColor

In [98]:
def truthVector(Filename):
    # Parses the filename, and compares it against the record of sensor position on cranes
    # inputs: filename
    # outputs: truth vector


    #Parsing the file name.  Assuming it is in the standard format
    sSensor = Filename[23]
    sDate = datetime.datetime.strptime('20'+Filename[10:21],"%Y%m%d-%H%M")

    mask = []

    i=0
    #loops through the known sensor movements, and creates a filter mask
    for spf in OutputVectors:
        
        startDate = datetime.datetime.strptime(str(spf[0])+str(spf[1]).zfill(2)+str(spf[2]).zfill(2)
            +str(spf[3]).zfill(2)+str(spf[4]).zfill(2),"%Y%m%d%H%M")
        #datetime.date(int(spf[0]), int(spf[1]), int(spf[2])) + datetime.timedelta(hours=spf[3]) + datetime.timedelta(minutes=spf[4])
        endDate = datetime.datetime.strptime(str(spf[5])+str(spf[6]).zfill(2)+str(spf[7]).zfill(2)
            +str(spf[8]).zfill(2)+str(spf[9]).zfill(2),"%Y%m%d%H%M")
        #datetime.date(int(spf[5]), int(spf[6]), int(spf[7])) + datetime.timedelta(hours=spf[8]) + datetime.timedelta(minutes=spf[9])
        
        if sDate >= startDate and sDate <= endDate and int(spf[10]) == int(sSensor):
            mask.append(True)
            i+=1
        else:
            mask.append(False)
        
    if i != 1: print('error ', i, Filename)

    results = OutputVectors[mask,11:]

    if i > 1: 
        print('Found Two ', Filename)
        results = results[0,:]
    #np.array(results)

    return results

In [99]:
#opens all files in a folder and returns the 3 axis acceleration in one array
# made to run in parallel

def getAcceleration(FileName):
    
        DataSet = np.genfromtxt(open(FileName,'r'), delimiter=',',skip_header=0)
        JustFileName = FileName.rsplit('/', 1)[-1]
        if FileName[-20:-16] == 'Gyro':
            return [False,FileName,False]
        else:
            if FileName[-6:-5] == 's':
                FileDate = FileName[-18:-7]
                sensor = FileName[-5:-4]
            elif FileName[-21:-16] == 'Accel':
                FileDate = FileName[-15:-4]
                sensor = 1
            else:
                FileDate = FileName[-20:-4]
                sensor = 1
            return [[FileDate, 'x',DataSet[:,2], sensor,JustFileName],[FileDate,'y',DataSet[:,3],sensor,JustFileName],[FileDate,'z',DataSet[:,4],sensor,JustFileName]]
    

In [100]:
def KalmanFilterDenoise(data, rate=1):

    #https://jamwheeler.com/college-productivity/how-to-denoise-a-1-d-signal-with-a-kalman-filter-with-python/
    #

    def oavar(data, rate, numpoints=30):

        x = np.cumsum(data)

        max_ratio = 1/9
        num_points = 30
        ms = np.unique(
            np.logspace(0, np.log10(len(x) * max_ratio), numpoints
           ).astype(int))        

        oavars = np.empty(len(ms))
        for i, m in enumerate(ms):
            oavars[i] = (
                (x[2*m:] - 2*x[m:-m] + x[:-2*m])**2
            ).mean() / (2*m**2)

        return ms / rate, oavars

    def ln_NKfit(ln_tau, ln_N, ln_K):
        tau = np.exp(ln_tau)
        N, K = np.exp([ln_N, ln_K])
        oadev = N**2 / tau + K**2 * (tau/3)
        return np.log(oadev)

    def get_NK(data, rate):
        taus, oavars = oavar(data, rate)

        ln_params, ln_varmatrix = (
            curve_fit(ln_NKfit, np.log(taus), np.log(oavars))
        )
        return np.exp(ln_params)    

    # Initialize state and uncertainty
    state = data[0]
    output = np.empty(len(data))

    #rate = 1 # We can set this to 1, if we're calculating N, K internally
    # N and K will just be scaled relative to the sampling rate internally
    dt = 1/rate

    N, K = get_NK(data, rate)

    process_noise = K**2 * dt
    measurement_noise = N**2 / dt

    covariance = measurement_noise

    for index, measurement in enumerate(data):
        # 1. Predict state using system's model

        covariance += process_noise

        # Update
        kalman_gain = covariance / (covariance + measurement_noise)

        state += kalman_gain * (measurement - state)
        covariance = (1 - kalman_gain) * covariance

        output[index] = state

    return output

In [101]:
#Inputs:
#folder1 = 'C:\\Users\\Hendrickson\\Desktop\\Phone Acceleration\\'
#folder2 = 'E:\\Documents\\Dan\\PhD\\Data Backup\\ASC Accel Pi\\Excel Versions\\'
#folder = "E:\\Documents\\Dan\\Port Work\\OneDrive\OneDrive - The Port of Virginia\\Shared with Everyone\\Crane Data\\"
#folder = "C:\\Users\\Dan\\Desktop\\TempData\\"


files = os.listdir(folder)

Groups = 1
GroupSize = 5


In [102]:
#Pre Calculations

num_cores = multiprocessing.cpu_count()-1

if np.shape(files)[0] < GroupSize:
    GroupSize = np.shape(files)[0]

if Groups !=0:
    files = random.sample(files,GroupSize*Groups)

loops = int(float(np.size(files))/float(GroupSize))+1
start = time.time()


In [103]:
AllAccels = Parallel(n_jobs=num_cores)(delayed(getAcceleration)(folder+file) for file in files)


In [104]:
MetaData = []  #np.asarray([],dtype=object)
DataOnlyMatrix = np.asarray([],dtype=object)
for j in range(np.shape(AllAccels)[0]):
    if AllAccels[j][0] == False :
        if AllAccels[j][1][4:9] =='Accel':
            print(j,AllAccels[j][1])
    else: 
        for k in range(3):
            MetaData.append([AllAccels[j][k][0], AllAccels[j][k][1], AllAccels[j][k][3], AllAccels[j][k][4]])
            if np.size(DataOnlyMatrix) == 0:
                    DataOnlyMatrix =np.matrix(AllAccels[j][k][2])
            else:
                    DataOnlyMatrix = np.concatenate((DataOnlyMatrix,np.matrix(AllAccels[j][k][2])),axis=0)

In [105]:
waveKalmaned = np.asarray([],dtype=object)
waveKalmaned = Parallel(n_jobs=num_cores)(delayed(KalmanFilterDenoise)(np.asarray(data).flatten()) for data in DataOnlyMatrix)
waveKalmaned = np.matrix(waveKalmaned)

DataOnlyMatrix = []

In [106]:
length = np.shape(waveKalmaned)[0]
justifier = np.ones((length, np.shape(waveKalmaned)[1]))
average = np.zeros(length)
for i in range(length):
    average[i]= np.average(waveKalmaned[i][:])
justifier = justifier.T * average.T
waveKalmaned = waveKalmaned - justifier.T

In [109]:
maxes = np.amax(waveKalmaned[:,500:], axis = 1)
mins = np.amin(waveKalmaned[:,500:], axis = 1)

Files = np.zeros(int(mins.size/3))
Keep = np.zeros(mins.size)
for i in range(mins.size):
    if i % 3 == 0:
        if maxes[i] > 0.01 and mins[i] < -0.01:
            Keep[i]=1
            Keep[i+1]=1
            Keep[i+2]=1
            print(i)
            
            Files[int(i/3)] = 1

Keep = np.array(Keep, dtype='bool')
Files = np.array(Files, dtype='bool')

0
3
6


In [111]:
waveKalmaned = waveKalmaned[Keep,:]

In [113]:
newFiles = []

for i in range(np.size(files)):
    if Files[i]:
        newFiles.append(files[i])
        


In [115]:
MovesLeft = np.size(newFiles)
ColorWaveletMoves = []
for i in range(MovesLeft):
    ThreeAxis = np.concatenate((np.concatenate(([waveKalmaned[i*3,], waveKalmaned[i*3+1]]),axis=1)
                                                       , waveKalmaned[i*3+2]),axis=1)
    ColorWaveletMoves.append(ThreeAxis)

In [116]:
np.shape(ColorWaveletMoves)

(3, 1, 180000)

/sciclone/home20/dchendrickson01/.conda/envs/testenv2/lib/python3.7/site-packages/scipy/optimize/minpack.py:834: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/sciclone/home20/dchendrickson01/.conda/envs/testenv2/lib/python3.7/site-packages/scipy/optimize/minpack.py:834: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


In [80]:
images = Parallel(n_jobs=num_cores)(delayed(getWvltImage)(image) for image in ColorWaveletMoves)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
Truth = []
for file in newFiles:
    Truth.append(truthVector(file))

In [29]:
np.shape(images[0])

IndexError: list index out of range

In [27]:
batch_size = GroupSize
img_height = np.shape(images[0])[0]
img_width = np.shape(images[0])[1]


IndexError: list index out of range

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[files[i]])
    plt.axis("off")
